# Using + and * with seqs

Usually (have not encountered otherwise yet) both + and * operands must be of the same seq type, and the result is concatenation to a new seq of the same type rather than modification. New object is created, and operands are completely unchanged.

Terminology reminder: The val that the operator operates on is called the operand. E.g.: >>> 2+3 5. Here, + is the operator. 2 and 3 are operands and 5 is the output of the operation.

'*' multiplies the sequence
'+' adds sequences

In [22]:
l = [1, 2, 3]
print(l * 5)
print(l + l)
print( 5 * 'abcd')

#Be careful with this approach, as the new inner lists refer to the same list -- see Ex2-13 in the next section
print([[1, 2, 3]] * 3) # inner lists [1, 2, 3] and tuples (1, 2, 3) are treated as indiv eles to be concatenated
print([(1, 2, 3)] * 3)
print(((1, 2, 3)) * 3) # concatenates same as (1, 2, 3) * 3
print(([1, 2, 3]) * 3) # concatenates same as [1, 2, 3] * 3
print([[[1]]] * 3) # here, [[1]] is the list element to be multiplied

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
[1, 2, 3, 1, 2, 3]
abcdabcdabcdabcdabcd
[[1, 2, 3], [1, 2, 3], [1, 2, 3]]
[(1, 2, 3), (1, 2, 3), (1, 2, 3)]
(1, 2, 3, 1, 2, 3, 1, 2, 3)
[1, 2, 3, 1, 2, 3, 1, 2, 3]
[[[1]], [[1]], [[1]]]


## Pitfalls of using * to initialize a list of lists
Best way to initialize a list w/ certain num of nested lists is w/ listcomps. e.g.: distribute students in list of teams, representing squares on a game board (tic-tac-toe, see Ex2-12 below)

In [21]:
board = [['_'] * 3 for i in range(3)] #for loop tells us this multiplication will occur 3 times when i = (0, 1, 2)
print(board)
board[1][2] = 'X' #replace ele at second index within the first index inner list with X
print(board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]


Ex2-13 depicts the wrong way to go about this (more in Ch8):

In [40]:
weird_board = [['_'] * 3] * 3
print(weird_board) # looks the same as in Ex2-12, but...
weird_board[1][2] = 'X'
print(weird_board) # the same replacement results in the second index of every inner list being affected

# To illustrate this further...
weird_board[0][2] = 'X' # replace second index of 0th index inner list
print(weird_board) # same as replacing 1st index inner list
weird_board[2][2] = 'X' # replace second index of 2nd index inner list
print(weird_board)  # same as replacing 1st index inner list

# What is actually happening in line 1 of this Ex2-13
row = ['_'] * 3 #referenced before loop
board = []
for i in range(3):
    board.append(row) #same row being appended
print("\nEx2-13:")
print(board)
board[0][2] = 'X'
print(board)

# Compared to Ex2-12
board = []
for i in range(3):
    row = ['_'] * 3 # new row referenced during each iteration
    board.append(row) # new row appended
print("\nEx2-12:")
print(board)
board[0][2] = 'X'
print(board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]
[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]
[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

Ex2-13:
[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

Ex2-12:
[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', 'X'], ['_', '_', '_'], ['_', '_', '_']]


## Augmented assignment with seqs using += and *=
+= special method is __iadd__ (in-place addition), but if __iadd__ isn't implemented, Python uses __add__

e.g. >>> a += b
With mutable seqs (list, bytearray, array.array), "a" will be changed in-place
- similar to a.extend(b)

When __iadd__ is not implemented, such as with immutable seqs, a = a + b, meaning a new "a" is being produced

These concepts apply to *=, special method __imul__. These will be further discussed in Ch13.

In [6]:
# mutable seq
l = [1, 2, 3]
print(f"{l}:", id(l)) # shows ID of the list
l *= 2
print(f"{l}:", id(l)) # shows ID of the list has not changed

# immutable seq
t = (1, 2, 3)
print(f"{t}:", id(t)) # ID once again
t *= 2
print(f"{t}:", id(t)) # ID is different this time, because this is a new t


[1, 2, 3]: 140317528769536
[1, 2, 3, 1, 2, 3]: 140317528769536
(1, 2, 3): 140316995146816
(1, 2, 3, 1, 2, 3): 140317528914144


## A += Assignment puzzler
Evaluating the following cell (Ex2-14) results in the item t[2] changing AND an exception being raised.

Why?

In [8]:
t = (1, 2, [30, 40])
try:
    t[2] += [50, 60]
except TypeError:
    print(t)

(1, 2, [30, 40, 50, 60])


In [10]:
#Using bytecode
import dis
dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


Will return to this when I better understand Top of Stack (TOS) and what each of these elements mean
- LOAD_NAME to BINARY_SUBSCR is setting up the value of s[a] on TOS
- LOAD_NAME to INPLACE_ADD is performing TOS += b (succeeds if mutable)
- ROT_THREE to STORE_SUBSCR assigns s[a] = TOS (fails if immutable, which is what we're dealing with with the "t" tuple)

Lessons:
1) Mutable items in tuples is not good
2) augmented assignment is not an atomic operation (what does this mean?*). It threw an exception after doing part of its job.
3) Python bytecode helps to see what is going on

*An operation is atomic if it cannot be interrupted